<a href="https://colab.research.google.com/github/eluisluzquadros/covid_impact_se/blob/main/step01_2_collection_white_ScraperGoogleDrive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Instalação das dependencias
#### SEMPRE RODAR ANTES

In [67]:

#!apt-get update # to update ubuntu to correctly run apt install

# scraper
!pip install selenium
!apt install chromium-chromedriver
!pip install webdriver-manager
!cp /usr/lib/chromium-browser/chromedriver/usr/bin

!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium msticpy tld folium
!pip pymongo pymongo[srv] dnspython

# geocoder ipaddress
!pip install msticpy

# get domain
!pip3 install tld


# spacy
!pip install -U pip setuptools wheel
!pip install -U spacy
#!python -m spacy download en
#!python -m spacy download pt
!python -m spacy download en_core_web_sm
# spacy contextual Spell Check
!pip install contextualSpellCheck
!curl https://raw.githubusercontent.com/codelucas/newspaper/master/download_corpora.py | python3




Reading package lists... Done
Building dependency tree       
Reading state information... Done
chromium-chromedriver is already the newest version (95.0.4638.69-0ubuntu0.18.04.1).
0 upgraded, 0 newly installed, 0 to remove and 52 not upgraded.
cp: missing destination file operand after '/usr/lib/chromium-browser/chromedriver/usr/bin'
Try 'cp --help' for more information.
Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:9 htt

### Import das dependencias

In [68]:
import os
import glob
import selenium
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
#from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime, date, timedelta

import urllib
import socket
from msticpy.sectools.geoip import IPStackLookup

import spacy
import contextualSpellCheck

import pymongo
import numpy as np
import pandas as pd
from datetime import datetime as dt
from datetime import date, timedelta
from pandas.tseries.offsets import DateOffset
import dns

import tld
from tld import get_tld, get_fld

from google.colab import drive

### Connect google drive

In [69]:
# Codigo para montar o Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [70]:
#Caminho no google drive
path = "/content/drive/My Drive/covid19impactSE/data/02_white/2021/01_collect/en/"

# Listar os arquivos da pasta no google drive
os.chdir(path)
!ls

2021-01-01.csv	2021-01-02.csv


### Import chaves de api

In [71]:
# drive
keys = pd.read_csv("/content/drive/My Drive/covid19impactSE/keys/credentials.csv", sep=';')
access_token_ipstack = keys.loc[0,"key"]
access_token_pymongo = keys.loc[1,"key"]

### Create functions

In [72]:
import pandas as pd
GSE_RESULTS_CLASS = 'gs_r gs_or gs_scl' #'ZINbbc xpd O9g5cc uUPGi'
#GSE_MAX_RESULTS = 200

NLP = spacy.load("en_core_web_sm")
contextualSpellCheck.add_to_pipe(NLP)

def get_driver():
  options = webdriver.ChromeOptions()
  options.add_argument('--headless')
  options.add_argument('--no-sandbox')
  options.add_argument('--disable-dev-shm-usage')
  options.add_argument('user-agent=Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/32.0.1667.0 Safari/537.36')    
  return webdriver.Chrome('chromedriver',options=options)

def page_source(query, date):
  start = 0
  language = 'en'
  as_ylo = 2020
  as_yhi = 2021
  url = "https://scholar.google.com/scholar?start={}&q={}&hl={}&as_sdt=0,5&as_ylo={}&as_yhi={}".format(start, keyword, language, as_ylo, as_yhi)
  print(url)
  #print('\n')
  driver = get_driver()
  driver.get(url)
  soup = BeautifulSoup(driver.page_source, 'html.parser') 
  return driver.page_source


  
def fetch_result(date, item):
  try:
    return {
        'date': date.strftime('%Y-%m-%d'),
        'header' : item.select_one('h3').text,
        'summary': item.findChildren('div', recursive=False)[-1].text,
        'link' : item.find('a', href=True)['href'].replace('/url?q=', '').split('&sa')[0],
            
    }
    
  except:
    return None

def search(query, date):
  soup = BeautifulSoup(page_source(query, date), 'html.parser') 
  html_results = soup.find_all('div', {'class': GSE_RESULTS_CLASS })
  results = [fetch_result(date, r) for r in html_results]
  return pd.DataFrame(list(filter(None, results)))

def fetch_domain(df):
  df['protocol'],df['domain'],df['path'],df['query'],df['fragment'] = zip(*[urllib.parse.urlsplit(x) for x in df.link])
  return df

def fetch_hostname(df): 
  urls = df['link']
  hostnames = []
  for url in urls:
    try:
      res = get_tld(url, as_object=True)
      hostnames.append(res.fld)
    except:
      res = np.nan
      hostnames.append(res)
  df['host_name'] = hostnames
  return df

def fetch_ipaddress(df):
  domains = df['domain']
  ipaddress = []
  for i in domains:
      i_strip = i.strip()
      try:
          ip = socket.gethostbyname(i_strip)
          ipaddress.append(ip)
      except socket.gaierror:
        ipaddress.append('127.0.0.1')
  df['ip_address'] = ipaddress
  return df

def fetch_location(df):
  iplocation = IPStackLookup(api_key=access_token_ipstack)
  df = iplocation.df_lookup_ip(df, column="ip_address")
  return df

def fetch_nlp(df):
  df['header_context_true'] = df['header'].apply(lambda x: str(NLP(x)))
  df['summary_context_true'] = df['summary'].apply(lambda x: str(NLP(x)))
  return df

def save(df):
  datamax = str(df['date'].max())
  fileName = path + datamax + '.csv'
  df.to_csv(fileName, sep=';', encoding='UTF-8')
  
def next_data(df):
  df['date'] = pd.to_datetime(df['date'], dayfirst = False)
  lst_date = df['date'].max()
  nxt_date = (lst_date.date() + DateOffset(days = 1)).date()
  return nxt_date

### APENAS PARA MONGODB ###
#def save(df):
#  client = pymongo.MongoClient(access_token_pymongo)
#  db = client.covid_impact
#  df = df.astype('str')
#  db.impacts.insert_many(df.to_dict('records'))

#def load():
#  client = pymongo.MongoClient(access_token_pymongo)
#  db = client.covid_impact
#  df = pd.DataFrame.from_records(db.impacts.find())
# return df

### Create strings

In [73]:
# Scraper 1 (2020)
keyword = '"covid impact" OR "disaster" OR "resilience" AND "software engineers" OR "software development" OR "disaster informatics" AND "productivity" OR "work from home" OR "remote working"'

# Scraper 3 (2021 v2)
# keyword = '"covid-19" OR "covid" OR "coronavirus" AND “impact" OR "disaster" OR "resilience" AND “software” OR "software engineer" OR "agile" OR "information industry" OR "IT" OR "disaster informatics" OR "crisis informatics" OR "productivity" OR "remote" OR "work" OR "future of work"'

### Merge csv's or SCRAPER new search

In [74]:
# merge multiples csv's files
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
files = len(all_filenames)

if files >= 1:
  #combine all files in path
  df = pd.concat([pd.read_csv(f, sep=';', parse_dates=["date"], engine="python", encoding='UTF-8') for f in all_filenames ])
else:
  #get first date
  first_date = date(2021, 1, 1)
  #get seach
  df = search(keyword,first_date)
  #get domain
  df = fetch_domain(df)
  #get hostname
  df = fetch_hostname(df)
  #get ipaddress
  ##df = fetch_ipaddress(df)
  #get geocoded ipaddress
  ##df = fetch_location(df)
  #save
  save(df)  

#print df
df.tail()

,Unnamed: 0,date,header,summary,link,protocol,domain,path,query,fragment,host_name,ip_address,CountryCode,CountryName,State,City,Longitude,Latitude,Asn,TimeGenerated,Type,AdditionalData,IpAddress,header_context_true,summary_context_true
11,11,2021-01-02,Pandemic programming,"Pandemic programmingR Paul, S Baltes, A Gianis...",https://search.proquest.com/openview/78d6fa772...,https,search.proquest.com,/openview/78d6fa7727a9ccc7d2bb47fb22fe717f/1,pq-origsite=gscholar&cbl=326341,NaN,proquest.com,165.215.200.239,US,United States,Michigan,Ann Arbor,-83.729179,42.223412,40675,NaN,geolocation,{},165.215.200.239,Pandemic programming,"Pandemic programmingR Paul, S Baltes, A Gianis..."
12,12,2021-01-02,[HTML][HTML] A topology of groups: What GitHub...,[HTML][HTML] A topology of groups: What GitHub...,https://www.sciencedirect.com/science/article/...,https,www.sciencedirect.com,/science/article/pii/S0040162520311173,NaN,NaN,sciencedirect.com,162.159.129.81,US,United States,California,San Francisco,-122.442497,37.769169,13335,NaN,geolocation,{},162.159.129.81,[HTML][HTML] A topology of groups: What GitHub...,[HTML][HTML] A topology of groups: What GitHub...
13,13,2021-01-02,[HTML][HTML] Predictors of well-being and prod...,[HTML][HTML] Predictors of well-being and prod...,https://link.springer.com/article/10.1007/s106...,https,link.springer.com,/article/10.1007/s10664-021-09945-9,NaN,NaN,springer.com,151.101.0.95,US,United States,California,San Francisco,-122.392860,37.767849,54113,NaN,geolocation,{},151.101.0.95,[HTML][HTML] Predictors of well-being and prod...,[HTML][HTML] Predictors of well-being and prod...
14,14,2021-01-02,[HTML][HTML] Predictors of well-being and prod...,[HTML][HTML] Predictors of well-being and prod...,https://link.springer.com/article/10.1007/s106...,https,link.springer.com,/article/10.1007/s10664-021-09945-9,NaN,NaN,springer.com,151.101.0.95,US,United States,California,San Francisco,-122.392860,37.767849,54113,NaN,geolocation,{},151.101.0.95,[HTML][HTML] Predictors of well-being and prod...,[HTML][HTML] Predictors of well-being and prod...
15,15,2021-01-02,[HTML][HTML] Predictors of well-being and prod...,[HTML][HTML] Predictors of well-being and prod...,https://link.springer.com/article/10.1007/s106...,https,link.springer.com,/article/10.1007/s10664-021-09945-9,NaN,NaN,springer.com,151.101.0.95,US,United States,California,San Francisco,-122.392860,37.767849,54113,NaN,geolocation,{},151.101.0.95,[HTML][HTML] Predictors of well-being and prod...,[HTML][HTML] Predictors of well-being and prod...


### SCRAPER new search

In [75]:
#get next data
next_date = next_data(df)

#get seach
df_new = search(keyword, next_date)

#get domain
df_new = fetch_domain(df_new)

#get hostname
df_new = fetch_hostname(df_new)

#get ipaddress
df_new = fetch_ipaddress(df_new)

#get geocoded ipaddress
df_new = fetch_location(df_new)

#get nlp contextualSpellCheck 
df_new = fetch_nlp(df_new)

#save df new
save(df_new)

#print new df
df_new.head(2)



https://scholar.google.com/scholar?start=0&q="covid impact" OR "disaster" OR "resilience" AND "software engineers" OR "software development" OR "disaster informatics" AND "productivity" OR "work from home" OR "remote working"&hl=en&as_sdt=0,5&as_ylo=2020&as_yhi=2021


,date,header,summary,link,protocol,domain,path,query,fragment,host_name,ip_address,CountryCode,CountryName,State,City,Longitude,Latitude,Asn,TimeGenerated,Type,AdditionalData,IpAddress,header_context_true,summary_context_true
0,2021-01-03,[PDF][PDF] Why we didn't conduct studies in so...,[PDF][PDF] Why we didn't conduct studies in so...,https://stride.org.uk/wp-content/uploads/2021/...,https,stride.org.uk,/wp-content/uploads/2021/07/ECSCW-2021_submitt...,,,stride.org.uk,195.216.197.42,GB,United Kingdom,England,Islington,-0.10000,51.533298,47625,None,geolocation,{},195.216.197.42,[PDF][PDF] Why we didn't conduct studies in so...,[PDF][PDF] Why we didn't conduct studies in so...
1,2021-01-03,[PDF][PDF] Quantifying Individual and Collecti...,[PDF][PDF] Quantifying Individual and Collecti...,https://www.research-collection.ethz.ch/bitstr...,https,www.research-collection.ethz.ch,/bitstream/handle/20.500.11850/515315/10/Abstr...,,,ethz.ch,129.132.8.151,CH,Switzerland,Zurich,Zürich,8.65443,47.384941,559,None,geolocation,{},129.132.8.151,[PDF][PDF] Quantifying Individual and Collecti...,[PDF][PDF] Quantifying Individual and Collecti...
